# Kaggle : 2 Sigma Connect : Rental Listing Inquiries

Purpose: To predict the interest level of a listing using the listing 
images only

Author : Chandan Panda 

Version: 1.0

In [1]:
from tqdm import tqdm
import numpy as np
import csv
import os
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from random import uniform
import bcolz
import time
from keras.layers import merge
from keras.layers.core import Lambda
from keras.models import Model
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras.layers import Input, Dense
import math
import pandas as pd
from keras.utils import np_utils as u
import utils; reload(utils)
from utils import *
import gc

%matplotlib inline
%cd '/cp/home/ubuntu/data'


Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


/cp/home/ubuntu/data


# Create datasets

We randomly divide the dataset into 5 folds and also keep a small sample dataset for debugging. Each fold contains about 54000 images

The images are structured into folders named high, medium or low based on the interest level of the listing in which the image was uploaded

We make sure that a listing's images are not scattered across both different folds!

In [ ]:
#Create 5 fold datasets

%cd '/cp/home/ubuntu/data/photos'
photos=glob("*.jpg")

CREATE_FOLDS = False
NumberOfFolds = 5.0

if CREATE_FOLDS==True:
    p = pd.read_json('../train.json')
    photos=glob("*.jpg")
    listings = list(set([f.split('_')[0] for f in photos]))
    shuffled_listings = np.random.permutation(listings)
    sample_size = int(round(len(listings) * (1/NumberOfFolds),0))
    
    for i in range(int(NumberOfFolds)): 
        os.mkdir('/cp/home/ubuntu/data/folds/fold_'+str(i))
        for listing in shuffled_listings[range(i*sample_size,(i*sample_size)+sample_size)]:
            pics=glob(listing+"_*.jpg")
            for pic in pics:
                os.rename(pic, '/cp/home/ubuntu/data/folds/'+'fold_'+str(i)+'/' + pic)

        #Move photos into directories based on class type 
        s = '/cp/home/ubuntu/data/folds/fold_' + str(i)
        os.chdir(s)
        photos=glob('*.jpg')
        os.mkdir('low')
        os.mkdir('medium')
        os.mkdir('high')
        for photo in photos:
            listing_id = photo.partition("_")[0]
            row = p.loc[p['listing_id'] == int(listing_id) ]
            interest_level = 'NA'
            if len(row)>0:
                interest_level = row['interest_level'].iloc[0]
                os.rename(photo, interest_level + '/' + photo)
        os.chdir('/cp/home/ubuntu/data/photos')

# Sample a few images 

In [ ]:
#To Be Done

# Initialize Pre-trained Network 

We obtain a version of the VGG network (a convolutional neural network by the Visual Geometry and Graphics group) that is trained of the ImageNet database to predict 1000 classes

This version includes batch normalization layers that ensure that the weights not not become too large and improves the training time. In addition, it normalizes the image by subtracting the RGB mean values from the ImageNet database

In [3]:
from vgg16bn import Vgg16BN
model = vgg_ft_bn(3)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

We will now split the VGG network into the convolutional and dense layers. This is so that we can reuse the convolutional layers weights as it is and train only the final three dense layers. This will save enormous amount of time while making little difference to our final performance

In [4]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
gen=image.ImageDataGenerator(width_shift_range = 0.2 , height_shift_range = 0.2 , shear_range = 0.2 , zoom_range = 0.2)
target_size=(224,224)
batch_size = 64
class_mode='categorical'
shuffle=True
dirname = 'train'
model.compile(optimizer=Adam(1e-3),loss='categorical_crossentropy', metrics=['accuracy'])
conv_layers,fc_layers = split_at(model, Convolution2D)
conv_model = Sequential(conv_layers)

# Generate Features

Preprocess all images in sample, training and validation sets by passing through the VGG network convolutional layers after a bit of data augmentation. These features (i.e. 512 X 14 X 14 per image) will then be used as inputs  to a dense layer network that finally outputs the probability of the image being low, medium or high interest_level

Because of the large number of images (~200k training set, ~70k in validation set) and RAM constraint of 64 GB, we will divide the features into chunks of 10k images and use the fit_generator and evaluate_generator to iterate through each file chunk as we train and validate the dense layer network.

Also, the images are randomly width shifted, height shifted, sheared and zoomed to introduce bias in order to reduce variance

Each epoch will take approximately an hour!

In [7]:
def generateFeatures(dirname,CHUNK_SIZE = 10000):
    GENERATE_FEATURES = True
    if GENERATE_FEATURES==True:
        batches = gen.flow_from_directory('/cp/home/ubuntu/data/folds/'+dirname, 
                                                         target_size=target_size,class_mode=class_mode, 
                                          shuffle=True, batch_size=1)
        labels_all = onehot(batches.classes) 
        totalImages = batches.nb_sample
        listing_ids_all = np.ndarray([totalImages,1],dtype=np.int64)
        listing_ids_all = [int((f.split('_')[0]).split('/')[1]) for f in batches.filenames]
        batchIndex = 0
        numImages = 0
        images = np.ndarray([CHUNK_SIZE,3, 224, 224],dtype=np.float32)
        labels = np.ndarray([CHUNK_SIZE,3],dtype=np.float64)
        listing_ids = np.ndarray([CHUNK_SIZE,1],dtype=np.int64)   
        for i in range(totalImages):
            if (numImages > (CHUNK_SIZE - 1)) or (i == (totalImages-1)):
                labels = labels_all[batchIndex*CHUNK_SIZE:batchIndex*CHUNK_SIZE + numImages]
                listing_ids = listing_ids_all[batchIndex*CHUNK_SIZE:batchIndex*CHUNK_SIZE + numImages]
                save_array('features/' + dirname + '_labels_%d.dat'%batchIndex,labels[0:numImages])               
                save_array('features/' + dirname + '_listing_ids_%d.dat'%batchIndex,listing_ids[0:numImages])                     
                conv_feat = conv_model.predict_generator(generator=gen.flow(images[0:numImages],batch_size=batch_size)
                                                         ,val_samples=numImages)        
                save_array('features/' + dirname + '_trainX_%d.dat'%batchIndex,conv_feat[0:numImages])  
                batchIndex += 1
                images = np.ndarray([CHUNK_SIZE,3, 224, 224],dtype=np.float32)
                labels = np.ndarray([CHUNK_SIZE,3],dtype=np.float64)
                numImages = 0
                del(conv_feat)
                del(labels)
                gc.collect()            
            x,y = batches.next()
            images[numImages] = x[0]
            numImages += 1

In [ ]:
generateFeatures('fold_0',CHUNK_SIZE=20000)
generateFeatures('fold_1',CHUNK_SIZE=20000)
generateFeatures('fold_2',CHUNK_SIZE=20000)
generateFeatures('fold_3',CHUNK_SIZE=20000)
generateFeatures('fold_4',CHUNK_SIZE=20000)

In [ ]:
generateFeatures('sample',CHUNK_SIZE=5000)

Move augmented features to another folder

In [ ]:
%cd '/cp/home/ubuntu/data/features'
photos=glob("*fold*.dat")
for photo in photos:
    os.rename(photo, '/cp/home/ubuntu/data/features-aug/'+photo)
%cd '/cp/home/ubuntu/data'

Function to randomize fold feature chunks. This is currently hardcoded to handle exactly 3 filechunks per fold!

In [6]:
def randomizeFeatures(fold,folder):
    os.chdir(folder)
    Files = glob(fold+'_trainX*.dat')
    chunk0 = load_array(Files[0])
    chunk1 = load_array(Files[1])
    chunk2 = load_array(Files[2])
    chunk0size = chunk0.shape[0]
    chunk1size = chunk1.shape[0]
    chunk2size = chunk2.shape[0]
    labels_chunk0 = load_array(Files[0].replace('trainX','labels'))
    labels_chunk1 = load_array(Files[1].replace('trainX','labels'))
    labels_chunk2 = load_array(Files[2].replace('trainX','labels'))
    listing_ids_chunk0 = load_array(Files[0].replace('trainX','listing_ids'))
    listing_ids_chunk1 = load_array(Files[1].replace('trainX','listing_ids'))
    listing_ids_chunk2 = load_array(Files[2].replace('trainX','listing_ids'))
    features = np.vstack((chunk0,chunk1,chunk2))
    del(chunk0)
    del(chunk1)
    del(chunk2)
    gc.collect()
    labels = np.vstack((labels_chunk0,labels_chunk1,labels_chunk2))
    del(labels_chunk0)
    del(labels_chunk1)
    del(labels_chunk2)
    gc.collect()
    listing_ids = np.concatenate((listing_ids_chunk0,listing_ids_chunk1,listing_ids_chunk2))
    del(listing_ids_chunk0)
    del(listing_ids_chunk1)
    del(listing_ids_chunk2)
    gc.collect()

    seq = np.random.permutation(range(features.shape[0]))

    tempArrayFeatures = np.ndarray([1,512, 14, 14],dtype=np.float32)
    tempArrayListingIds = 0
    tempArrayLabels = np.ndarray([1,3],dtype=np.float32)

    for i in range(features.shape[0]):
        tempArrayFeatures[0] = features[seq[i]]
        features[seq[i]] = features[i]
        features[i] = tempArrayFeatures[0]
        tempArrayListingIds = listing_ids[seq[i]]
        listing_ids[seq[i]] = listing_ids[i]
        listing_ids[i] = tempArrayListingIds
        tempArrayLabels[0] = labels[seq[i]]
        labels[seq[i]] = labels[i]
        labels[i] = tempArrayLabels[0]

    save_array(Files[0],features[range(chunk0size)])
    save_array(Files[1],features[range(chunk0size,chunk0size+chunk1size)])
    save_array(Files[2],features[range(chunk0size+chunk1size,features.shape[0])])
    save_array(Files[0].replace('trainX','labels'),labels[range(chunk0size)])
    save_array(Files[1].replace('trainX','labels'),labels[range(chunk0size,chunk0size+chunk1size)])
    save_array(Files[2].replace('trainX','labels'),labels[range(chunk0size+chunk1size,features.shape[0])])
    save_array(Files[0].replace('trainX','listing_ids'),listing_ids[range(chunk0size)])
    save_array(Files[1].replace('trainX','listing_ids'),listing_ids[range(chunk0size,chunk0size+chunk1size)])
    save_array(Files[2].replace('trainX','listing_ids'),listing_ids[range(chunk0size+chunk1size,features.shape[0])])
    os.chdir('/cp/home/ubuntu/data')
    return

In [11]:
def checkLabels(folder):    
    os.chdir(folder)
    chunks=glob("fold_*_labels_*.dat")
    gc.collect()
    for chunk in chunks:
        labels = load_array(chunk)
        print(chunk,np.sum(labels[:,0]),np.sum(labels[:,1]),np.sum(labels[:,2]))
    os.chdir('/cp/home/ubuntu/data')
    return

In [6]:
checkLabels('/cp/home/ubuntu/data/features-aug')

(0.0, 20000.0, 0.0)
(0.0, 1098.0, 12921.0)
(0.0, 1428.0, 12412.0)
(0.0, 1076.0, 12578.0)
(4459.0, 15541.0, 0.0)
(0.0, 1282.0, 12909.0)
(0.0, 20000.0, 0.0)
(0.0, 1111.0, 13174.0)
(0.0, 20000.0, 0.0)
(4304.0, 15696.0, 0.0)
(0.0, 20000.0, 0.0)
(4358.0, 15642.0, 0.0)
(4152.0, 15848.0, 0.0)
(0.0, 20000.0, 0.0)
(4376.0, 15624.0, 0.0)


In [13]:
randomizeFeatures('fold_0','/cp/home/ubuntu/data/features-aug')
randomizeFeatures('fold_1','/cp/home/ubuntu/data/features-aug')
randomizeFeatures('fold_2','/cp/home/ubuntu/data/features-aug')
randomizeFeatures('fold_3','/cp/home/ubuntu/data/features-aug')
randomizeFeatures('fold_4','/cp/home/ubuntu/data/features-aug')

In [14]:
checkLabels('/cp/home/ubuntu/data/features-aug')

('fold_2_labels_1.dat', 1503.0, 13037.0, 5460.0)
('fold_4_labels_2.dat', 1150.0, 9249.0, 3620.0)
('fold_1_labels_2.dat', 1453.0, 8807.0, 3580.0)
('fold_0_labels_2.dat', 981.0, 7311.0, 5362.0)
('fold_0_labels_0.dat', 1328.0, 13337.0, 5335.0)
('fold_3_labels_2.dat', 1192.0, 9329.0, 3670.0)
('fold_3_labels_1.dat', 741.0, 14978.0, 4281.0)
('fold_2_labels_2.dat', 441.0, 7969.0, 5875.0)
('fold_0_labels_1.dat', 2150.0, 15969.0, 1881.0)
('fold_2_labels_0.dat', 2360.0, 15801.0, 1839.0)
('fold_1_labels_1.dat', 1067.0, 14188.0, 4745.0)
('fold_3_labels_0.dat', 2425.0, 12617.0, 4958.0)
('fold_1_labels_0.dat', 1632.0, 14281.0, 4087.0)
('fold_4_labels_1.dat', 792.0, 14920.0, 4288.0)
('fold_4_labels_0.dat', 2434.0, 12553.0, 5013.0)


Create a dense layer network with batch normalization to reduce the time it takes to train. Also include a drop out of 0.4 to improve the validation accuracy. This network takes the pre-processed features from the saved numpy arrays and outputs 3 probabilities - interest level will be low, medium or high

In [5]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1),
        Dropout(p/4),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(3, activation='softmax')
    ]

# Train the Network

Training a network on such a large dataset requires us to use the generator functions in keras. These functions loop 
indefinitely and return to the yield statement the next time the function is called. This allows us to iterate through the 
image features one batch at a time till all of them are passed through the network. Once a numpy array is completed, the next 
one is loaded from memory so that at any given time, a maximum of 20k images are in memory.

The metrics such as validation categorical cross-entropy loss and accuracy are logged to a log file so that even if 
we lose the connection midway, we can review how the metrics changed across the epochs

Finally, we keep saving the best model's weights

In [5]:
def trainMyNw(nb_epoch,fold,folder):
    
    myGen=image.ImageDataGenerator()
    best_loss_metric = 999

    for e in tqdm(range(nb_epoch)):
        
        start = time.clock()
        f = open('/cp/home/ubuntu/data/results/log.txt', 'a')       
        featureFiles = glob(folder + '/' + 'fold_*_trainX_*.dat')        
        chunkCount=0
        
        for files in featureFiles:
            if files.find(fold) == -1:
                features = load_array(files)
                labels = load_array(files.replace('trainX','labels'))
                chunkCount += 1
                numImages = features.shape[0]
                bn_model.fit_generator(generator=myGen.flow(features,labels),samples_per_epoch = numImages,nb_epoch=1,verbose=0)  
                del(features)
                del(labels)
                gc.collect()
            
        featureFiles = glob(folder + '/' + fold + '_trainX_*.dat')        
        accuracy = 0
        loss = 0
        avg_accuracy = 0
        avg_loss = 0
        chunkCount = 0
        totalImages = 0

        for files in featureFiles:
            features = load_array(files)
            labels = load_array(files.replace('trainX','labels'))
            chunkCount += 1
            numImages = features.shape[0]
            loss_metrics = bn_model.evaluate_generator(generator=myGen.flow(features,labels),
                                                       val_samples = numImages) 
            accuracy += (loss_metrics[1]*numImages)
            loss += (loss_metrics[0]*numImages)
            totalImages += numImages
            del(features)
            del(labels) 
            gc.collect()            
            
        if chunkCount>0: 
            avg_accuracy = accuracy/totalImages
            avg_loss = loss/totalImages
            
        print bn_model.metrics_names,avg_loss,avg_accuracy, "time :", time.clock() - start
        f.write(str(bn_model.metrics_names))
        f.write(" : ")
        f.write(str(avg_loss))
        f.write(" : ")
        f.write(str(avg_accuracy))
        f.write(" : time :")
        f.write(str(time.clock() - start))
        f.write("\n")        
        f.close() 
        if (avg_loss<best_loss_metric):
            bn_model.save_weights('/cp/home/ubuntu/data/results/best.hdf5') 
            best_loss_metric = avg_loss
            
    return 

We train the network with a high learning rate once and then gradually decrease the learning rate as we go through about 12 epochs

A dropout of 0.2 is optimal for training on train dataset while a dropout of 0.4 suits the valid dataset. Train the network twice, once on the training and once on the validation dataset and save the versions with the lowest loss for later use for stacking

Save best model weights

In [ ]:
p=0.9
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
trainMyNw(1,'fold_0','features-aug')
bn_model.optimizer.lr=0.001
trainMyNw(1,'fold_0','features-aug')
os.rename('/cp/home/ubuntu/data/results/best.hdf5', '/cp/home/ubuntu/data/results/fold_0_XXXXn.hdf5')

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
p=0.9
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
trainMyNw(1,'fold_1','features-aug')
bn_model.optimizer.lr=0.001
trainMyNw(1,'fold_1','features-aug')
os.rename('/cp/home/ubuntu/data/results/best.hdf5', '/cp/home/ubuntu/data/results/fold_1_XXXXn.hdf5')

In [ ]:
p=0.9
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
trainMyNw(1,'fold_2','features-aug')
bn_model.optimizer.lr=0.001
trainMyNw(1,'fold_2','features-aug')
os.rename('/cp/home/ubuntu/data/results/best.hdf5', '/cp/home/ubuntu/data/results/fold_2_XXXXn.hdf5')

In [ ]:
p=0.9
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
trainMyNw(1,'fold_3','features-aug')
bn_model.optimizer.lr=0.001
trainMyNw(1,'fold_3','features-aug')
os.rename('/cp/home/ubuntu/data/results/best.hdf5', '/cp/home/ubuntu/data/results/fold_3_XXXXn.hdf5')

In [ ]:
p=0.9
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
trainMyNw(1,'fold_4','features-aug')
bn_model.optimizer.lr=0.001
trainMyNw(1,'fold_4','features-aug')
os.rename('/cp/home/ubuntu/data/results/best.hdf5', '/cp/home/ubuntu/data/results/fold_4_XXXXn.hdf5')

Train final model on entire dataset. Manually set the number of epochs based on analysis of training logs

In [ ]:
p=0.9
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
trainMyNw(1,'sample','features-aug')
bn_model.optimizer.lr=0.001
trainMyNw(1,'sample','features-aug')

Save model weights

In [ ]:
bn_model.save_weights('/cp/home/ubuntu/data/results/fold_all_XXXXn.hdf5') 

Create a folder with photos for entire dataset

# Generate features for final scoring

Generate features for Test Dataset

In [ ]:
gen=image.ImageDataGenerator()
generateFeatures('test',CHUNK_SIZE=20000)

Generate features for all folds without data augmentation

In [ ]:
gen=image.ImageDataGenerator()
generateFeatures('fold_0',CHUNK_SIZE=20000)
generateFeatures('fold_1',CHUNK_SIZE=20000)
generateFeatures('fold_2',CHUNK_SIZE=20000)
generateFeatures('fold_3',CHUNK_SIZE=20000)
generateFeatures('fold_4',CHUNK_SIZE=20000)

In [ ]:
randomizeFeatures('fold_0','/cp/home/ubuntu/data/features')
randomizeFeatures('fold_1','/cp/home/ubuntu/data/features')
randomizeFeatures('fold_2','/cp/home/ubuntu/data/features')
randomizeFeatures('fold_3','/cp/home/ubuntu/data/features')
randomizeFeatures('fold_4','/cp/home/ubuntu/data/features')

In [ ]:
checkLabels('/cp/home/ubuntu/data/features')

# Score Datasets

In [ ]:
def scoreMyData(fold,weights_path):
    gen=image.ImageDataGenerator()
    bn_model.load_weights(weights_path) 
    featureFiles = glob('features/' + fold + '_trainX_*.dat')        

    for files in featureFiles:
        features = load_array(files)
        numImages = features.shape[0]
        scores = np.ndarray([numImages,3],dtype=np.float64)
        scores = bn_model.predict_generator(generator=gen.flow(features),val_samples = numImages)       
        save_array('predictions/' + files.split('/')[1].replace('trainX','scores'),scores)               
        del(scores)
        del(features)
        gc.collect()            
    return 

In [ ]:
scoreMyData('sample','/cp/home/ubuntu/data/results/fold_0_XXXXn.hdf5')

Score all folds

In [ ]:
scoreMyData('fold_0','/cp/home/ubuntu/data/results/fold_0_XXXXn.hdf5')
scoreMyData('fold_1','/cp/home/ubuntu/data/results/fold_1_XXXXn.hdf5')
scoreMyData('fold_2','/cp/home/ubuntu/data/results/fold_2_XXXXn.hdf5')
scoreMyData('fold_3','/cp/home/ubuntu/data/results/fold_3_XXXXn.hdf5')
scoreMyData('fold_4','/cp/home/ubuntu/data/results/fold_4_XXXXn.hdf5')

Score test dataset

In [ ]:
scoreMyData('test','/cp/home/ubuntu/data/results/fold_all_XXXXn.hdf5')

In [ ]:
#scoreMyData('test','/cp/home/ubuntu/data/results/fold_all_XXXXnE.hdf5')

# Create final Predictions file extract

In [40]:
%cd '/cp/home/ubuntu/data/predictions'
scoreFiles = glob('*scores*.dat') 
listing_ids_all = load_array('/cp/home/ubuntu/data/features/' + scoreFiles[0].replace('scores','listing_ids'))
scores_all = load_array(scoreFiles[0])

for i in range(2,len(scoreFiles)):
    files =  scoreFiles[i]
    listing_ids = load_array('/cp/home/ubuntu/data/features/' + files.replace('scores','listing_ids'))
    scores = load_array(files)
    listing_ids_all = np.concatenate((listing_ids_all, listing_ids))
    scores_all = np.vstack((scores_all, scores))

listing_ids_all.shape,scores_all.shape  

import pandas as pd 

df = pd.DataFrame(listing_ids_all)
df.to_csv("listing_ids_all.csv")

df = pd.DataFrame(scores_all)
df.to_csv("scores_all.csv")


/cp/home/ubuntu/data/predictions


# That's all Folks !

# Scrapbook

In [5]:
features1 = load_array('/cp/home/ubuntu/data/backup/fold_2_trainX_2.dat')
labels1 = load_array('/cp/home/ubuntu/data/backup/fold_2_labels_2.dat')
features2 = load_array('/cp/home/ubuntu/data/backup/fold_3_trainX_0.dat')
labels2 = load_array('/cp/home/ubuntu/data/backup/fold_3_labels_0.dat')

features = np.vstack((features1,features2))
labels = np.vstack((labels1,labels2))

del(features1)
del(features2)
del(labels1)
del(labels2)
gc.collect()

363

In [6]:
features1 = load_array('/cp/home/ubuntu/data/backup/fold_3_trainX_2.dat')
labels1 = load_array('/cp/home/ubuntu/data/backup/fold_3_labels_2.dat')
features2 = load_array('/cp/home/ubuntu/data/backup/fold_4_trainX_0.dat')
labels2 = load_array('/cp/home/ubuntu/data/backup/fold_4_labels_0.dat')

features_val = np.vstack((features1,features2))
labels_val = np.vstack((labels1,labels2))

del(features1)
del(features2)
del(labels1)
del(labels2)
gc.collect()

0

In [7]:
features.shape,features_val.shape,labels.shape,labels_val.shape

((34285, 512, 14, 14), (34191, 512, 14, 14), (34285, 3), (34191, 3))

In [8]:
p=0.2
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)

Train on 34285 samples, validate on 34191 samples
Epoch 1/5
34285/34285 [==============================] - 35s - loss: 1.2586 - acc: 0.4036 - val_loss: 1.0881 - val_acc: 0.4511
Epoch 2/5
34285/34285 [==============================] - 35s - loss: 0.8116 - acc: 0.6285 - val_loss: 1.1434 - val_acc: 0.4557
Epoch 3/5
34285/34285 [==============================] - 35s - loss: 0.5783 - acc: 0.7634 - val_loss: 1.2395 - val_acc: 0.4393
Epoch 4/5
34285/34285 [==============================] - 35s - loss: 0.3923 - acc: 0.8544 - val_loss: 1.4213 - val_acc: 0.4406
Epoch 5/5
34285/34285 [==============================] - 35s - loss: 0.2707 - acc: 0.9028 - val_loss: 1.5513 - val_acc: 0.4352


In [10]:
p=0.5
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)

Train on 34285 samples, validate on 34191 samples
Epoch 1/5
34285/34285 [==============================] - 35s - loss: 1.3512 - acc: 0.3978 - val_loss: 1.0769 - val_acc: 0.4746
Epoch 2/5
34285/34285 [==============================] - 35s - loss: 1.0894 - acc: 0.4825 - val_loss: 1.0597 - val_acc: 0.4758
Epoch 3/5
34285/34285 [==============================] - 35s - loss: 0.9715 - acc: 0.5382 - val_loss: 1.0775 - val_acc: 0.4721
Epoch 4/5
34285/34285 [==============================] - 35s - loss: 0.8729 - acc: 0.5968 - val_loss: 1.1066 - val_acc: 0.4663
Epoch 5/5
34285/34285 [==============================] - 36s - loss: 0.7713 - acc: 0.6519 - val_loss: 1.1655 - val_acc: 0.4613


In [15]:
p=0.5
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=10,validation_data=(features_val, labels_val),verbose=1)

Train on 34285 samples, validate on 34191 samples
Epoch 1/10
34285/34285 [==============================] - 38s - loss: 1.6478 - acc: 0.3414 - val_loss: 1.2229 - val_acc: 0.3798
Epoch 2/10
34285/34285 [==============================] - 37s - loss: 1.4212 - acc: 0.3881 - val_loss: 1.1560 - val_acc: 0.4120
Epoch 3/10
34285/34285 [==============================] - 38s - loss: 1.2844 - acc: 0.4212 - val_loss: 1.1244 - val_acc: 0.4324
Epoch 4/10
34285/34285 [==============================] - 38s - loss: 1.1798 - acc: 0.4660 - val_loss: 1.1036 - val_acc: 0.4475
Epoch 5/10
34285/34285 [==============================] - 38s - loss: 1.1180 - acc: 0.4881 - val_loss: 1.0920 - val_acc: 0.4526
Epoch 6/10
34285/34285 [==============================] - 38s - loss: 1.0726 - acc: 0.5082 - val_loss: 1.0859 - val_acc: 0.4567
Epoch 7/10
34285/34285 [==============================] - 38s - loss: 1.0293 - acc: 0.5279 - val_loss: 1.0887 - val_acc: 0.4592
Epoch 8/10
34285/34285 [==============================

In [14]:
p=0.5
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)

Train on 34285 samples, validate on 34191 samples
Epoch 1/5
34285/34285 [==============================] - 37s - loss: 1.1336 - acc: 0.4375 - val_loss: 0.9996 - val_acc: 0.4619
Epoch 2/5
34285/34285 [==============================] - 38s - loss: 0.9857 - acc: 0.4908 - val_loss: 1.0133 - val_acc: 0.4622
Epoch 3/5
34285/34285 [==============================] - 38s - loss: 0.9182 - acc: 0.5436 - val_loss: 1.0677 - val_acc: 0.4368
Epoch 4/5
34285/34285 [==============================] - 38s - loss: 0.8018 - acc: 0.6241 - val_loss: 1.1420 - val_acc: 0.4427
Epoch 5/5
34285/34285 [==============================] - 38s - loss: 0.6687 - acc: 0.7027 - val_loss: 1.2121 - val_acc: 0.4456


In [16]:
p=0.5
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.005), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)

Train on 34285 samples, validate on 34191 samples
Epoch 1/5
34285/34285 [==============================] - 38s - loss: 1.1263 - acc: 0.4453 - val_loss: 1.0356 - val_acc: 0.4365
Epoch 2/5
34285/34285 [==============================] - 38s - loss: 1.0014 - acc: 0.4743 - val_loss: 1.0264 - val_acc: 0.4690
Epoch 3/5
34285/34285 [==============================] - 38s - loss: 0.9480 - acc: 0.5198 - val_loss: 1.0167 - val_acc: 0.4585
Epoch 4/5
34285/34285 [==============================] - 38s - loss: 0.8927 - acc: 0.5627 - val_loss: 1.0601 - val_acc: 0.4641
Epoch 5/5
34285/34285 [==============================] - 38s - loss: 0.8191 - acc: 0.6144 - val_loss: 1.0887 - val_acc: 0.4453


In [17]:
p=0.5
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)

Train on 34285 samples, validate on 34191 samples
Epoch 1/5
34285/34285 [==============================] - 38s - loss: 1.1241 - acc: 0.4491 - val_loss: 0.9942 - val_acc: 0.4857
Epoch 2/5
34285/34285 [==============================] - 39s - loss: 0.9839 - acc: 0.4798 - val_loss: 0.9933 - val_acc: 0.4868
Epoch 3/5
34285/34285 [==============================] - 39s - loss: 0.9652 - acc: 0.5009 - val_loss: 0.9978 - val_acc: 0.4682
Epoch 4/5
34285/34285 [==============================] - 39s - loss: 0.9444 - acc: 0.5224 - val_loss: 1.0237 - val_acc: 0.4795
Epoch 5/5
34285/34285 [==============================] - 39s - loss: 0.9197 - acc: 0.5425 - val_loss: 1.0683 - val_acc: 0.4532


In [18]:
p=0.6
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)

Train on 34285 samples, validate on 34191 samples
Epoch 1/5
34285/34285 [==============================] - 39s - loss: 1.1307 - acc: 0.4367 - val_loss: 1.0019 - val_acc: 0.4667
Epoch 2/5
34285/34285 [==============================] - 38s - loss: 0.9970 - acc: 0.4808 - val_loss: 1.0224 - val_acc: 0.4678
Epoch 3/5
34285/34285 [==============================] - 39s - loss: 0.9492 - acc: 0.5229 - val_loss: 1.0390 - val_acc: 0.4644
Epoch 4/5
34285/34285 [==============================] - 40s - loss: 0.8755 - acc: 0.5709 - val_loss: 1.0864 - val_acc: 0.4644
Epoch 5/5
34285/34285 [==============================] - 38s - loss: 0.7756 - acc: 0.6411 - val_loss: 1.1546 - val_acc: 0.4361


In [19]:
p=0.7
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)

Train on 34285 samples, validate on 34191 samples
Epoch 1/5
34285/34285 [==============================] - 39s - loss: 1.1506 - acc: 0.4445 - val_loss: 0.9887 - val_acc: 0.4944
Epoch 2/5
34285/34285 [==============================] - 39s - loss: 0.9860 - acc: 0.4784 - val_loss: 0.9870 - val_acc: 0.4942
Epoch 3/5
34285/34285 [==============================] - 38s - loss: 0.9779 - acc: 0.4870 - val_loss: 0.9966 - val_acc: 0.4621
Epoch 4/5
34285/34285 [==============================] - 39s - loss: 0.9752 - acc: 0.4909 - val_loss: 0.9944 - val_acc: 0.4895
Epoch 5/5
34285/34285 [==============================] - 40s - loss: 0.9756 - acc: 0.4947 - val_loss: 1.0008 - val_acc: 0.4808


In [20]:
p=0.7
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=1,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr = 0.001
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=1,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr = 0.00001
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=3,validation_data=(features_val, labels_val),verbose=1)

Train on 34285 samples, validate on 34191 samples
Epoch 1/1
34285/34285 [==============================] - 39s - loss: 1.1373 - acc: 0.4461 - val_loss: 1.0036 - val_acc: 0.4945
Train on 34285 samples, validate on 34191 samples
Epoch 1/1
34285/34285 [==============================] - 38s - loss: 0.9870 - acc: 0.4788 - val_loss: 0.9851 - val_acc: 0.4879
Train on 34285 samples, validate on 34191 samples
Epoch 1/3
34285/34285 [==============================] - 39s - loss: 0.9787 - acc: 0.4857 - val_loss: 0.9991 - val_acc: 0.4275
Epoch 2/3
34285/34285 [==============================] - 38s - loss: 0.9740 - acc: 0.4908 - val_loss: 1.0190 - val_acc: 0.4870
Epoch 3/3
34285/34285 [==============================] - 40s - loss: 0.9761 - acc: 0.4958 - val_loss: 1.0172 - val_acc: 0.4784


In [21]:
p=0.7
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=1,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr = 0.00001
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=3,validation_data=(features_val, labels_val),verbose=1)

Train on 34285 samples, validate on 34191 samples
Epoch 1/1
34285/34285 [==============================] - 39s - loss: 1.1497 - acc: 0.4456 - val_loss: 0.9969 - val_acc: 0.4871
Train on 34285 samples, validate on 34191 samples
Epoch 1/3
34285/34285 [==============================] - 39s - loss: 0.9859 - acc: 0.4784 - val_loss: 0.9833 - val_acc: 0.4898
Epoch 2/3
34285/34285 [==============================] - 39s - loss: 0.9775 - acc: 0.4873 - val_loss: 0.9856 - val_acc: 0.4934
Epoch 3/3
34285/34285 [==============================] - 39s - loss: 0.9670 - acc: 0.5015 - val_loss: 1.0088 - val_acc: 0.4462


In [22]:
p=0.1
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr = 0.001
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=1,validation_data=(features_val, labels_val),verbose=1)

Train on 34285 samples, validate on 34191 samples
Epoch 1/5
34285/34285 [==============================] - 38s - loss: 1.4684 - acc: 0.3474 - val_loss: 1.3273 - val_acc: 0.3513
Epoch 2/5
34285/34285 [==============================] - 38s - loss: 1.0632 - acc: 0.4852 - val_loss: 1.2647 - val_acc: 0.3787
Epoch 3/5
34285/34285 [==============================] - 38s - loss: 0.8376 - acc: 0.6158 - val_loss: 1.2392 - val_acc: 0.4015
Epoch 4/5
34285/34285 [==============================] - 38s - loss: 0.6795 - acc: 0.7191 - val_loss: 1.2428 - val_acc: 0.4116
Epoch 5/5
34285/34285 [==============================] - 38s - loss: 0.5465 - acc: 0.7958 - val_loss: 1.2675 - val_acc: 0.4203
Train on 34285 samples, validate on 34191 samples
Epoch 1/1
34285/34285 [==============================] - 39s - loss: 0.4460 - acc: 0.8484 - val_loss: 1.2969 - val_acc: 0.4199


In [23]:
p=0.9
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=1,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr=0.001
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=2,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr=0.0001
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=3,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr=0.00001
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=3,validation_data=(features_val, labels_val),verbose=1)

Train on 34285 samples, validate on 34191 samples
Epoch 1/1
34285/34285 [==============================] - 39s - loss: 1.1670 - acc: 0.4458 - val_loss: 0.9929 - val_acc: 0.4849
Train on 34285 samples, validate on 34191 samples
Epoch 1/2
34285/34285 [==============================] - 39s - loss: 0.9887 - acc: 0.4749 - val_loss: 0.9796 - val_acc: 0.4945
Epoch 2/2
34285/34285 [==============================] - 39s - loss: 0.9862 - acc: 0.4786 - val_loss: 0.9877 - val_acc: 0.4923
Train on 34285 samples, validate on 34191 samples
Epoch 1/3
34285/34285 [==============================] - 39s - loss: 0.9967 - acc: 0.4677 - val_loss: 0.9964 - val_acc: 0.4935
Epoch 2/3
34285/34285 [==============================] - 40s - loss: 1.0036 - acc: 0.4626 - val_loss: 0.9880 - val_acc: 0.4872
Epoch 3/3
34285/34285 [==============================] - 39s - loss: 1.0037 - acc: 0.4626 - val_loss: 1.0039 - val_acc: 0.4867
Train on 34285 samples, validate on 34191 samples
Epoch 1/3
34285/34285 [===============

In [ ]:
gc.collect()

In [ ]:
import inspect
print inspect.getsource(bn_model.evaluate_generator)